Allison Aprile<br>
CS-584 A<br>
March 1, 2021<br>
HW1<br>

# **Preprocessing**

In [28]:
# Imports
import tarfile
import pandas as pd
import numpy as np
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from google.colab import drive
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.utils import to_categorical

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Extract data from tar.gz file (then uploaded to Google Drive)
'''
# Extract data from tar.gz data
tar = tarfile.open('/Users/aaprile/Desktop/ag_news_csv.tar.gz')
tar.extractall()
tar.close()
'''

"\n# Extract data from tar.gz data\ntar = tarfile.open('/Users/aaprile/Desktop/ag_news_csv.tar.gz')\ntar.extractall()\ntar.close()\n"

In [6]:
# Mount Google Drive
drive.mount("/content/drive")

Mounted at /content/drive


## <font color=navy> **Load data**

In [34]:
# Import training dataset
train = pd.read_csv('/content/drive/My Drive/train.csv', header=None)

# Rename columns
train.columns = ['class', 'title', 'description']

# Add 'all_text' column (combine text of title and description)
train['all_text'] = train['title'] + ' ' + train['description']

# Confirm
print(train.shape)
train.head()

(120000, 4)


,class,title,description,all_text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."


In [35]:
# Import testing dataset
test = pd.read_csv('/content/drive/My Drive/test.csv', header=None)

# Rename columns
test.columns = ['class', 'title', 'description']

# Add 'all_text' column (combine text of title and description)
test['all_text'] = test['title'] + ' ' + test['description']

# Confirm
print(test.shape)
test.head()

(7600, 4)


,class,title,description,all_text
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...,Fears for T N pension after talks Unions repre...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o...",The Race is On: Second Private Team Sets Launc...
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...,Ky. Company Wins Grant to Study Peptides (AP) ...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...,Prediction Unit Helps Forecast Wildfires (AP) ...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...,Calif. Aims to Limit Farm-Related Smog (AP) AP...


## <font color=navy> **Preprocess data**

In [10]:
# Define text cleaning function to apply to 'title' and 'description' columns
def preprocess_text(text):
    # Given string 'text':
    
    # Normalize text by making all lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove whitespace and split into tokens
    tokens = text.split()
    
    # Define stopwords
    stop_words = stopwords.words()
    
    # Remove any nonalphabetic, stopwords, common words, or single letter words
    words = [tok for tok in tokens if tok.isalpha() and tok not in stop_words
            and tok not in ['reuters', 'site', 'afp', 'ap'] and len(tok) > 1]

    # Return text joined by spaces
    return " ".join(words)

In [14]:
# Process training and testing data
train_vals = [preprocess_text(train['all_text'][i]) for i in range(len(train))]
test_vals = [preprocess_text(test['all_text'][j]) for j in range(len(test))]

In [36]:
# Add processed columns to dataframes
train['processed_text'] = train_vals
test['processed_text'] = test_vals

In [37]:
# Confirm
train.head()

,class,title,description,all_text,processed_text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...,wall st bears claw back black short sellers wa...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...,carlyle looks toward commercial aerospace priv...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...,oil economy cloud stocks outlook soaring crude...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...,iraq halts oil exports main southern pipeline ...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new...",oil prices soar time record posing new menace ...


In [38]:
test.head()

,class,title,description,all_text,processed_text
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...,Fears for T N pension after talks Unions repre...,fears pension talks unions representing worker...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o...",The Race is On: Second Private Team Sets Launc...,race second private team sets launch date huma...
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...,Ky. Company Wins Grant to Study Peptides (AP) ...,ky company wins grant study peptides company f...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...,Prediction Unit Helps Forecast Wildfires (AP) ...,prediction unit helps forecast wildfires barel...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...,Calif. Aims to Limit Farm-Related Smog (AP) AP...,calif aims limit farm related smog southern ca...


In [39]:
# Write to CSV file (for future loading)
train.to_csv('/content/drive/My Drive/train_processed.csv', index=False)
test.to_csv('/content/drive/My Drive/test_processed.csv', index=False)

## <font color=navy>**Feature selection**

In [19]:
# Get top 2000 words per class
def get_top_2000(df, cls):
  # Get data
  class_data = list(df[df['class'] == cls]['processed_text'].values)

  # Define and fit Count Vectorizer
  count_vec = CountVectorizer().fit(class_data)

  # Save words and word counts
  words = count_vec.get_feature_names()
  counts = count_vec.transform(class_data)

  # Calculate document frequency
  doc_freq = np.squeeze(np.asarray(counts.sum(axis=0)))

  # Get top indices
  ind = sorted(range(len(doc_freq)), key=lambda i: doc_freq[i])[-2000:]

  # Return top 2000 words
  return [words[i] for i in ind]

In [24]:
# Define vocabulary, given top 2000 words per class
vocab = get_top_2000(train, 1) + get_top_2000(train, 2) + get_top_2000(train, 3) + get_top_2000(train, 4)

# Remove duplicates
vocab = list(set(vocab))

# Print length of final vocabulary
print(len(vocab))

4576


In [25]:
# Write vocabulary to text file
with open('/content/drive/My Drive/vocabulary.txt', 'w') as f:
    for item in vocab:
        f.write("%s\n" % item)

## <font color=navy>**Construct examples**

In [26]:
# Define TfidfVectorizer object using formulated vocabulary
vectorizer = TfidfVectorizer(vocabulary=vocab)

In [29]:
# Create training feature vector and label vector (and encode labels)
X_train = vectorizer.fit_transform(list(train['processed_text'].values)).toarray()
y_train = train['class'].values
y_train_oh = to_categorical(y_train)[:, -4:]

# Confirm shapes
print(X_train.shape)
print(y_train.shape)
print(y_train_oh.shape)

(120000, 4576)
(120000,)
(120000, 4)


In [40]:
# Create testing feature vector and label vector (and encode labels)
X_test = vectorizer.fit_transform(list(test['processed_text'].values)).toarray()
y_test = test['class'].values
y_test_oh = to_categorical(y_test)[:, -4:]

# Confirm shapes
print(X_test.shape)
print(y_test.shape)
print(y_test_oh.shape)

(7600, 4576)
(7600,)
(7600, 4)


## <font color=navy>**Save data**

In [32]:
# Save training data
np.save('/content/drive/My Drive/X_train.npy', X_train)
np.save('/content/drive/My Drive/y_train.npy', y_train)
np.save('/content/drive/My Drive/y_train_oh.npy', y_train_oh)

In [33]:
# Save testing data
np.save('/content/drive/My Drive/X_test.npy', X_test)
np.save('/content/drive/My Drive/y_test.npy', y_test)
np.save('/content/drive/My Drive/y_test_oh.npy', y_test_oh)
